### import
- **设置日志级别**

In [36]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import urllib.request
import numpy as np
import tensorflow as tf

# 设置日志级别
tf.logging.set_verbosity(tf.logging.INFO)

### get data

In [37]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"
IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
    raw = urllib.request.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.request.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
        f.write(raw)

# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING, target_dtype=np.int, features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST, target_dtype=np.int, features_dtype=np.float32)

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

### dataset utils
由于数据量很小，可以把数据定义在常量中

In [38]:
# Define the training inputs
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    return x, y

# Define the test inputs
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    return x, y

# Classify two new flower samples.
def new_samples():
    return np.array([[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

### train and test
- **用ValidationMonitor每隔N步在测试集上验证一次准确率**
    1. 首先定义validation_monitor
    2. 在fit时加上monitors=[validation_monitor]

- **ValidationMonitor的使用依赖于checkpoints,初始化classifier时用`config = tf.contrib.learn.RunConfig(save_checkpoints_secs=1)`配置tf每1秒保存一下checkpoints**

- **可以自定义log中显示的validation_metrics**
- **让训练提前终止**
```
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200
```

In [39]:
# 定义log中要显示的validation_metrics
validation_metrics = {
    "accuracy":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_accuracy,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES),
    "precision":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_precision,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES),
    "recall":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_recall,
            prediction_key=tf.contrib.learn.PredictionKey.CLASSES)
}

# 用ValidationMonitor每隔N步在测试集上验证一次准确率
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics, 
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200
)

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3, 
    model_dir="/tmp/iris_model",
    config = tf.contrib.learn.RunConfig(save_checkpoints_secs=1)
)

# Fit model.
classifier.fit(input_fn=get_train_inputs, steps=2000, monitors=[validation_monitor])

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs, steps=1)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

predictions = list(classifier.predict(input_fn=new_samples))
print("New Samples, Class Predictions: {}\n".format(predictions))

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_save_checkpoints_secs': 1, '_save_checkpoints_steps': None, '_task_id': 0, '_session_config': None, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_save_summary_steps': 100, '_is_chief': True, '_num_ps_replicas': 0, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020682D91BA8>, '_model_dir': '/tmp/iris_model', '_task_type': None, '_master': '', '_keep_checkpoint_max': 5}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary 

INFO:tensorflow:Saving dict for global step 101: accuracy = 0.966667, global_step = 101, loss = 0.154192, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 150): global_step = 101, accuracy = 0.966667, recall = 1.0, precision = 1.0, loss = 0.154192
INFO:tensorflow:Saving checkpoints for 151 into /tmp/iris_model\model.ckpt.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-25-03:05:31
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-301
INFO:tensorflow:Finished evaluation at 2017-06-25-03:05:31
INFO:tensorflow:Saving dict for global step 301: accuracy = 0.966667, global_step = 301, loss = 0.0621895, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 350): global_step = 301, accuracy = 0.966667, recall = 1.0, precision = 1.0, loss = 0.0621895
INFO:tensorflow:Saving checkpoints for 351 into /tmp/iris_model\model.ckpt.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and bat

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-25-03:05:36
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-501
INFO:tensorflow:Finished evaluation at 2017-06-25-03:05:37
INFO:tensorflow:Saving dict for global step 501: accuracy = 0.966667, global_step = 501, loss = 0.0529991, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 550): 

INFO:tensorflow:Saving dict for global step 651: accuracy = 0.966667, global_step = 651, loss = 0.0512633, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 700): global_step = 651, accuracy = 0.966667, recall = 1.0, precision = 1.0, loss = 0.0512633
INFO:tensorflow:Saving checkpoints for 701 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:global_step/sec: 35.6537
INFO:tensorflow:step = 701, loss = 0.0578337 (2.803 sec)
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Est

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-06-25-03:05:46
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-851
INFO:tensorflow:Finished evaluation at 2017-06-25-03:05:47
INFO:tensorflow:Saving dict for global step 851: accuracy = 0.966667, global_step = 851, loss = 0.0510519, precision = 1.0, recall = 1.0
INFO:tensorflow:Validation (step 900): global_step = 851, accuracy = 0.966667, recall = 1.0, precision = 1.0, loss = 0.0510519
INFO:tensorflow:Saving checkpoints for 901 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:global_step/sec: 34.6531
INFO:tensorflow:step = 901, loss = 0.0543923 (2.886 sec)
Instructions for updating:
Estimator is de